In [1]:
import numpy as np
import copy
import random
import sys
from tqdm import tqdm

In [2]:
world_height, world_width = 5, 10
epsilon = 0.0000001
alpha = 0.2
gamma = 0.5
epochs = 10000
start_pos, end_pos = (world_height-1, 0), (world_height-1, world_width-1)

In [3]:
def next_action(pos, policy, epsilon=0) -> int:
    validActionList = [0, 1, 2, 3]
    if pos[0]==0:
        validActionList.remove(0)
    if pos[0]==world_height-1:
        validActionList.remove(1)
    if pos[1]==0:
        validActionList.remove(2)
    if pos[1]==world_width-1:
        validActionList.remove(3)

    if np.random.uniform(0, 1) <= epsilon:
        return random.sample(validActionList, 1)[0]
    else:
        # check validity
        if np.argmax(policy[pos[0]][pos[1]]) in validActionList:
            return np.argmax(policy[pos[0]][pos[1]])
        else:
            return random.sample(validActionList, 1)[0]
        
# 这里不check是否在cliff导致返回start_pos
def next_pos(pos, action) -> tuple:
    if action == 0:
        return (pos[0]-1, pos[1])
    if action == 1:
        return (pos[0]+1, pos[1])
    if action == 2:
        return (pos[0], pos[1]-1)
    if action == 3:
        return (pos[0], pos[1]+1)
    
def get_reward(pos) -> int:
    if pos == end_pos:
        return -1
    if pos[0] == world_height-1:
        return -100
    return -1

def show_policy(policy):
    current_pos = start_pos
    episode = [current_pos]
    while current_pos != end_pos:
        step_action = next_action(current_pos, policy)
        step_pos = next_pos(current_pos, step_action)
        if step_pos[0] == world_height-1 and 0 < step_pos[1] < world_width-1:
            step_pos = start_pos
        current_pos = step_pos
        if current_pos in episode:
            print("fail\n")
            break
        episode.append(current_pos)
    if current_pos == end_pos:
        for pos in episode[:-1]:
            print(pos, end='->')
        pos = episode[-1]
        print(pos)
    

In [4]:
def Sarsa(epochs):
    Q_matrix = [[[0.0 for _ in range(4)] for _ in range(world_width)] for _ in range(world_height)]
    for row in range(world_height):
        for column in range(world_width):
            if row==0:
                Q_matrix[row][column][0] = -sys.maxsize - 1
            if row==world_height-1:
                Q_matrix[row][column][1] = -sys.maxsize - 1
            if column==0:
                Q_matrix[row][column][2] = -sys.maxsize - 1
            if column==world_width-1:
                Q_matrix[row][column][3] = -sys.maxsize - 1
    for epoch in range(epochs):
        current_pos = start_pos
        while current_pos != end_pos:
            step_action = next_action(current_pos, Q_matrix, epsilon)
            step_pos = next_pos(current_pos, step_action)
            step_reward = get_reward(step_pos)
            if step_pos[0] == world_height-1 and 0 < step_pos[1] < world_width-1:
                step_pos = start_pos
            forsee_action = next_action(step_pos, Q_matrix, epsilon)
            Q_matrix[current_pos[0]][current_pos[1]][step_action] += alpha * (step_reward + gamma * Q_matrix[step_pos[0]][step_pos[1]][forsee_action] - Q_matrix[current_pos[0]][current_pos[1]][step_action])
            current_pos = step_pos
    return Q_matrix

def Q_learning(epochs):
    Q_matrix = [[[0.0 for _ in range(4)] for _ in range(world_width)] for _ in range(world_height)]
    for row in range(world_height):
        for column in range(world_width):
            if row==0:
                Q_matrix[row][column][0] = -sys.maxsize - 1
            if row==world_height-1:
                Q_matrix[row][column][1] = -sys.maxsize - 1
            if column==0:
                Q_matrix[row][column][2] = -sys.maxsize - 1
            if column==world_width-1:
                Q_matrix[row][column][3] = -sys.maxsize - 1
    for epoch in range(epochs):
        current_pos = start_pos
        while current_pos != end_pos:
            step_action = next_action(current_pos, Q_matrix, epsilon)
            step_pos = next_pos(current_pos, step_action)
            step_reward = get_reward(step_pos)
            if step_pos[0] == world_height-1 and 0 < step_pos[1] < world_width-1:
                step_pos = start_pos
            forsee_action = next_action(step_pos, Q_matrix)
            Q_matrix[current_pos[0]][current_pos[1]][step_action] += alpha * (step_reward + gamma * Q_matrix[step_pos[0]][step_pos[1]][forsee_action] - Q_matrix[current_pos[0]][current_pos[1]][step_action])
            current_pos = step_pos
    return Q_matrix

In [5]:
if __name__ == "__main__":
    Sarsa_policy = Sarsa(epochs)
    show_policy(Sarsa_policy)
    Q_learning_policy = Q_learning(epochs)
    show_policy(Q_learning_policy)
    

(4, 0)->(3, 0)->(3, 1)->(3, 2)->(3, 3)->(3, 4)->(3, 5)->(3, 6)->(3, 7)->(3, 8)->(3, 9)->(4, 9)
(4, 0)->(3, 0)->(3, 1)->(3, 2)->(3, 3)->(3, 4)->(3, 5)->(3, 6)->(3, 7)->(3, 8)->(3, 9)->(4, 9)
